In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from data_import import preprocess_import_data

In [18]:
df = pd.concat([pd.read_excel('./scan_data/202109_all_scan_data.xlsx'), 
                pd.read_excel('./scan_data/202110_all_scan_data.xlsx'),
                pd.read_excel('./scan_data/202111_all_scan_data.xlsx'),
                pd.read_excel('./scan_data/202112_all_scan_data.xlsx')])

preprocess_import_data(df)
df.head(3)

,age,gender,height_cm,weight_kg,adult_child,department,hospital_ward,scan_area,scan_method,scan_protocol,scan_series,kV,mA,rotation_time,CTDI
0,72,M,170.0,83.0,成人,救急科,外来,胸部〜骨盤CT,造影,5.7 P+CE Chest-Pelvis Routine,1: Plain,100.0,366.41,0.5,16.64
1,72,M,170.0,83.0,成人,救急科,外来,胸部〜骨盤CT,造影,5.7 P+CE Chest-Pelvis Routine,2: CE,100.0,366.41,0.5,16.61
2,85,M,171.0,58.9,成人,循環器内科,外来,胸部〜骨盤CT,単純,5.7 P+CE Chest-Pelvis Routine,1: Plain,120.0,234.59,0.5,16.66


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBRegressor, XGBRFRegressor

In [4]:
oe = OrdinalEncoder()
oe.set_output(transform='pandas')
cat_cols = df.select_dtypes(exclude=np.number).columns.to_list()
df[cat_cols] = oe.fit_transform(df[cat_cols])

df.dropna(inplace=True)

target = 'CTDI'
X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [5]:
xgb_reg = XGBRegressor(learning_rate=0.01,
                       n_estimators=100,
                       max_depth=3)

In [6]:
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [8]:
y_pred = xgb_reg.predict(X_test)

In [9]:
print(f'mean_absolte_error: {mean_absolute_error(y_test, y_pred)}')
print(f'mean_absolute_percentage_error: {mean_absolute_percentage_error(y_test, y_pred)}')

mean_absolte_error: 7.1140329639247195
mean_absolute_percentage_error: 0.3441726022405118


In [12]:
df_predict = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})

df_predict['diff'] = df_predict['y_test'] - df_predict['y_pred']

df_predict

,y_test,y_pred,diff
898,18.68,9.344996,9.335004
1645,27.50,22.073439,5.426561
1918,19.13,11.260633,7.869367
104,15.44,9.242926,6.197074
2367,21.86,16.102682,5.757318
...,...,...,...
1841,34.38,22.226583,12.153417
538,10.22,6.642324,3.577676
2239,11.46,6.358378,5.101622
1888,6.60,5.651766,0.948234


## XGBRFRegressor

In [13]:
xgbrf = XGBRFRegressor(learning_rate=0.1,
                       n_estimators=100,
                       max_depth=3,
                       objective='reg:squarederror')

In [14]:
xgbrf.fit(X_train, y_train)

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.1, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=3, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=100, n_jobs=None, num_parallel_tree=None,
               objective='reg:squarederror', predictor=None, random_state=None, ...)

In [15]:
y_pred = xgbrf.predict(X_test)

In [16]:
print(f'mean_absolte_error: {mean_absolute_error(y_test, y_pred)}')
print(f'mean_absolute_percentage_error: {mean_absolute_percentage_error(y_test, y_pred)}')

mean_absolte_error: 16.808149725765478
mean_absolute_percentage_error: 0.8529620157541723


In [17]:
df_predict = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})
df_predict['diff'] = df_predict['y_test'] - df_predict['y_pred']
df_predict

,y_test,y_pred,diff
898,18.68,1.944909,16.735091
1645,27.50,3.979474,23.520526
1918,19.13,1.978924,17.151076
104,15.44,1.945466,13.494534
2367,21.86,3.096879,18.763121
...,...,...,...
1841,34.38,3.990477,30.389523
538,10.22,1.428334,8.791666
2239,11.46,1.404023,10.055977
1888,6.60,1.372123,5.227877
